# Face Detection and Image Similarity Search

## Introduction
When the [face detection workflow](https://docs.aperturedata.io/workflows/face_detection) is executed, it detects faces in images stored in ApertureDB and adds bounding boxes around them. 
If the "Extract embeddings" option is selected, embeddings are generated for each detected face bounding box.
In this notebook, we’ll use the detected face bounding boxes and their embeddings to search for images containing similar faces.

For comprehensive details on bounding boxes, polygons, and other region-based features in ApertureDB, please refer to [ApertureDB Image documentation](https://docs.aperturedata.io/category/image-and-related-commands). 
For details on the Image interface in the ApertureDB python SDK, please refer to the [python SDK documentation](https://docs.aperturedata.io/python_sdk/object_wrappers/Images).

## Setup
Before running this notebook, make sure the following prerequisites are met:

1. Images in ApertureDB
   Ensure your ApertureDB cloud instance contains images. You can ingest images using supported workflows such as the "[Ingest_from_Bucket](https://docs.aperturedata.dev/workflows/ingest_from_bucket)" which allows importing data directly from AWS or GCP storage.
2. Face Detection with Embeddings
   Run the [face detection workflow](https://docs.aperturedata.io/workflows/face_detection) with "Extract embeddings" option enabled.
   This step is essential for performing similarity searches based on facial features.


## Import some modules we will need

In [ ]:
from aperturedb.CommonLibrary import create_connector, execute_query
from aperturedb.NotebookHelpers import display as display_images
from aperturedb.Images import Images

## Set up client connection

In [ ]:
# This will only work if you have apererturedb installed and configured.
# The configuration is either created by setting an APERTUREDB_KEY environment variable,
# or by creating a configuration using adb config.
# See https://docs.aperturedata.io/Setup/client/adb for more information.
client = create_connector()

# If you wish to explicitly use the Connector class, you can do so like this:
#from aperturedb import Connector as Connector
#client = Connector.Connector(host="<YOUR_HOST_NAME_HERE>", user="<YOUR_USERNAME_HERE>", password="<YOUR_PASSWORD_HERE>")

response, _ = client.query([{"GetStatus": {}}])
print(response)


## Retrieve the unique id of the target face boundingbox

After running the face detection workflow, each image is assigned a unique **wf_sha1_hash** value, which can be used as an image identifier. 
Using the ApertureDB Web UI, you can view both the images and their associated metadata properties.

To locate the bounding box for a specific image:
1. Search for the image using appropriate **constraints** based on metadata or hash values.
2. Use the **FindBoundingBox** command with the **is_connected_to** parameter to retrieve bounding boxes associated with the target images.
3. If multiple bounding boxes are returned, refer to the **coordinates**, **width**, and **height** to identify the bounding box of interest.
4. Extract the **_uniqueid** of the target bounding box for downstream tasks.


In [ ]:

display_limit = 5
query = [ {
        "FindImage": {
            "_ref": 1,
            "blobs": True,
            "constraints": {
                "wf_sha1_hash" : ["==", "<YOUR_WF_SHA1_HASH>"]
            },
            "results": {
                "all_properties": True
            }
        }      
    }, {
        "FindBoundingBox": {            
             "is_connected_to": {
                "ref": 1
            },
            "with_label": "face",
            "labels": True,
            "coordinates": True,
            "uniqueids": True
        }
    }
   ]

response, blobs = client.query(query)
print(response[1])

imgs = Images(client, batch_size= display_limit, response=response[0]["FindImage"]["entities"])
imgs.display( show_bboxes= True)


## Retrieving the Descriptor(Embedding) for the Target Face Bounding Box

After identifying the **_uniqueid** of the bounding box corresponding to the target face, use the **FindDescriptor** command with the **is_connected_to** parameter to retrieve its associated descriptor(embedding).

In [ ]:
query = [ {
        "FindBoundingBox": {
            "_ref": 1,
            "constraints": {
                "_uniqueid" : ["==", "1.211.12620"]
            }
        }      
    }, {
        "FindDescriptor": {
             "is_connected_to": {
                "ref": 1
            },
            "blobs": True,
            "results": {
                "all_properties": True
            }
        }
    }
   ]

response, face_blobs = client.query(query)
print(response[1])
# print(face_blobs[0])

## Find images containing a person with a similar face.
Once we’ve identified the descriptor corresponding to the face we want to search for, we’ll use it to perform a similarity search and find photos containing similar faces.
The descriptors generated through the face detection workflow are stored in the **wf_facenet_processed** DescriptorSet.
Using the **FindDescriptor** command, we’ll search for the 5 nearest neighbors of the given descriptor within the wf_facenet_processed DescriptorSet. We’ll then retrieve and display the bounding boxes and images associated with those descriptors.
For more details on FindDescriptor, please refer to the [Descriptor Commands documentation](https://docs.aperturedata.dev/query_language/Reference/descriptor_commands/desc_commands/FindDescriptor).

In [ ]:
query = [ {
        "FindDescriptor": {
            "_ref": 1,
            "set": "wf_facenet_processed",
            "k_neighbors": 5,
            "distances": True,
        }      
    }, {
        "FindBoundingBox": {
            "_ref" :2,
            "blobs": True,
             "is_connected_to": {
                "ref": 1
            },
            "results": {
                "all_properties": True
            }
        }
    }, {
        "FindImage": {
            "blobs": True,
            "is_connected_to": {
                "ref": 2
            },
            "results": {
                "list": ["wf_sha1_hash"]
            }
        }      
    }]
response, blobs = client.query(query, face_blobs)
#print(response)

bbox_blobs = blobs[:5]
img_blobs = blobs[5:]
print("face bounding boxes")
display_images(bbox_blobs)

print("images")
display_images(img_blobs)
